In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
#default framework for applications
import param, panel as pn, holoviews as hv, numpy as np, pandas as pd, hvplot.pandas
hv.extension('bokeh')
renderer = hv.renderer('bokeh')
pn.extension()
print('start')
#Read datasets
# #use temp storage to experiment
sub1 = pd.read_hdf('oordeel2018.h5',key='o2018')
#         0         1               2            3         4            5       6       7       8        9         10
colors=['red'   ,'gold'      , 'yellow', 'lightgreen', 'blue', 'white', 'white','lightsalmon','lightskyblue','red'   , 'blue']
omap= { 'oordeel':{ 'slecht':0,'ontoereikend':1,'matig':2,'goed':3,'zeer goed':4,'voldoet niet':9,'voldoet':10} }

localdir='datasets/'
datafiles= ['4.monitoringprogramma_owl_20181016.csv']
for index in range(len(datafiles)):
    locd=pd.read_csv(localdir+datafiles[index], sep=';', decimal='.', dtype=object, encoding='latin1')
    locd.columns = locd.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('.', '_').str.replace(')', '')

merged=pd.merge(sub1,locd,left_on=['waterlichaam_identificatie','typering_omschrijving'],
                right_on=['representatief_waterlichaam_code','parameter_omschrijving'], how='right')

#remove biological oordeel, select 2018 and mark projections 
merged=merged.dropna(subset=['typering_omschrijving'])
#merged=merged.query('rapportagejaar=="2018"')
merged['projectie']=merged['waterlichaam_identificatie']!=merged['ligt_in_waterlichaam_code'] 

#set lists for selectors
startup=True
class dataPlot(param.Parameterized):
    
    #definition of widget-based-parameters
    beheerder = param.ObjectSelector(default="Waterschap Scheldestromen", objects=sorted(list(merged.waterbeheerder_omschrijving.unique())))
    #stroomgebied = param.ObjectSelector(default='all', objects=['all']+list(merged.stroomgebieddistrict_code.unique()))
    
    #dataselection 
    def get_data(self):
        sub1=merged.copy()
        sub1=sub1.query('waterbeheerder_omschrijving==@self.beheerder')
        #converteer oordeel naar numerieke waarde
        sub1=sub1.replace(omap)
        #split dataset by projectie
        split1, split2 = [x for _, x in sub1.groupby(sub1['projectie'])]
        
        sub3=split2.query('oordeel<8').copy() #use this only for chemicals so split others to sub3 /KEEP
        sub2=split2.query('oordeel>8').copy() #the chemicals, replace oordeel where projected with oordeel-2
        sub2['oordeel'] = sub2['oordeel']-2 
        sub2=pd.concat([sub2,sub3,split1]).drop_duplicates() #recombine dataset and remove duplicates
        #create dataset for heatmap, select max per wl, partially projected will become NOT projected 
        subs=sub2.groupby(['waterlichaam_identificatie','typering_omschrijving','waterbeheerder_omschrijving'])
        sub2=pd.DataFrame(subs['oordeel'].max()).reset_index()
        
        return sub2

    def save(self):
        pass
    
    
    @param.depends('beheerder', watch=True) #watch reacts to any change of paramA or param1
    def create_plot(self):
        df = self.get_data()
        low=df['oordeel'].min() #find lowest value to allow slicing the cmap-dictionary
        cats=df.typering_omschrijving.unique() #coun
        pheight=cats.shape[0]*20+50
        if startup:
            pheight=1000
        p=df.hvplot.heatmap(colorbar=False,ylabel='',rot=90,flip_yaxis=True, width=1200, height=pheight,
                            x='waterlichaam_identificatie' , y='typering_omschrijving',
                            C='oordeel',  hover_cols=['waterbeheerder_omschrijving'],
                            cmap=colors[low:] ).opts(xaxis='top',yaxis='right', xlabel='', )
        #renderer.save(p, 'test')
        return p

dataplot = dataPlot(name='')

#external widget
button = pn.widgets.Button(name='Click me')
def on_click(event):
    viewplot.paramA='d'
button.param.watch(on_click, 'clicks')

#paramcontrols can be called like this to allow carefull building of the gui
plt=pn.panel(dataplot.create_plot)
right = pn.panel(dataplot, parameters=['beheerder'], width=150)
#left = pn.panel(dataplot, parameters=['stroomgebied'], width=150)
desc = pn.pane.HTML("""
    <br><h3>Monitoringprogramma en oordelen</h3><br>
    Overzicht van monitoring van chemie, gebaseerd op WKP gegevens uit 2018.
    Het oordeel voor chemische stoffen is rood/blauw indien gemeten, lichtere tinten rood/blauw 
    geven aan waar gegevens uitsluitend op projectie zijn gebaseerd.
    """, width=450)

# pn.Column(desc,
#           pn.Row(left,right), 
#           plt).servable()
pn.Column(pn.Row(desc,pn.Column(pn.Spacer(height=60),pn.Row(pn.Spacer(width=50), right))),
          pn.Spacer(height=20),
          plt
                 ).servable()